In this notebook we write the code for sorting transitions timeseries.

## Loading Data

In [1]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm.notebook import tqdm



In [ ]:
raw_data_dir = '/Users/cfn18/Desktop/Double-Well-SR/Test-Transition-Data/'

def nc_test(file_name):
    return file_name[-3:] == '.nc'

def nc_file_list(d):
    nc_list = []
    for file in os.listdir(d):
        if nc_test(file):
            nc_list.append(d + file)
    return nc_list

integration_files = nc_file_list(raw_data_dir)

In [ ]:
ds_file = integration_files[0]
ds = xr.open_mfdataset(ds_file)

## Step 1: Tag points in timeseries near fixed points

In [4]:
def _tag_cold_points(ts, ball_size=0.1):
    return -(np.sqrt((ts.x + 1 )**2 + ts.y **2) < ball_size).values.astype(int)

def _tag_hot_points(ts, ball_size=0.1):
    return (np.sqrt((ts.x - 1 )**2 + ts.y **2) < ball_size).values.astype(int)

def symbolic_ts(ts, ball_size=0.1):
    """
    Identifies all points in a timeseries with -1, 0 or 1.
    -1 means you're close to (-1, 0)
    1 means you're close to (1, 0)
    0 is rest of points.
    """
    cold_points = _tag_cold_points(ts, ball_size)
    hot_points = _tag_hot_points(ts, ball_size)
    return cold_points + hot_points

In [5]:
# Functions take a single timeseries and return transition from hot to cold or vice versa

def cold_to_hot_transitions(ts, ball_size=0.1):
    c2h = [] # List of transitions
    symbolic_path = symbolic_ts(ts, ball_size)
    
    # Identifying sequences of the form (-1, 0, ..., 0, 1)
    for i in np.where(symbolic_path == -1)[0]:
        next_hot_point = np.argmax(symbolic_path[i:])
        
        # Is it only 0's between the -1 and 1?
        if (np.sum(symbolic_path[slice(i, i + next_hot_point + 1)]) == 0): 
            transition = ts.isel(time=slice(i, i + next_hot_point + 1))
            c2h.append(transition)
    return c2h

def hot_to_cold_transitions(ts, ball_size=0.1):
    h2c = [] # List of transitions
    symbolic_path = symbolic_ts(ts, ball_size)
    
    # Identifying sequences of the form (1, 0, ..., 0, -1)
    for i in np.where(symbolic_path == 1)[0]:
        next_cold_point = np.argmin(symbolic_path[i:])
        
        # Is it only 0's between the 1 and -1?
        if (np.sum(symbolic_path[slice(i, i + next_cold_point + 1)]) == 0): 
            transition = ts.isel(time=slice(i, i + next_cold_point + 1))
            h2c.append(transition)
    return h2c

In [ ]:
# TO DO: 
# Script opening of all datasets
# Save all transitions in appropriate places
# Arrange this to run in a cluster

ball_size=0.1

c2h = []
h2c = []
for r in ds.realisation:
    ts = ds.sel(realisation=r).drop('realisation')
    c2h += cold_to_hot_transitions(ts, ball_size)
    h2c += hot_to_cold_transitions(ts, ball_size)

In [ ]:
times = []
for ts in c2h:
    time = ts.time.max() - ts.time.min()
    times.append(time.item())

In [ ]:
plt.hist(np.log(times), density=True, bins=100)

In [8]:
# Opening datasets


alphas = [0.]
epsilons = [10., 1.]
ae_pairs = alpha_eps_pairs(alphas, epsilons)

# Use array jobs to decide input
# alpha, eps = alpha_eps_pairs[int(sys.argv[1]) - 1]


In [7]:
# Opening dataset

def integrations_parent_dir(cluster=False):
    if cluster:
        return '/rds/general/user/cfn18/ephemeral/Rotated-2D-Well-Stochastic-Model/'
    else: # on mac
        return '/Users/cfn18/Desktop/Double-Well-SR/Test-Transition-Data/'

def alpha_eps_pairs(alphas, epsilons):
    alpha_eps_pairs = []
    for alpha in alphas:
        for eps in epsilons:
            alpha_eps_pairs.append((alpha, eps))
    return alpha_eps_pairs

def integration_directories(alpha, eps, cluster=False):
    p_dir = integrations_parent_dir(cluster)
    alpha_sub_dir = f'alpha_{alpha}/'.replace('.', '_')
    eps_sub_dir = f'eps_{eps}/'.replace('.', '_')
    transition_directory = p_dir + alpha_sub_dir + eps_sub_dir
    return [transition_directory + x for x in ['cold-ensemble/', 'hot-ensemble/']]
    

In [9]:
def list_directory(d):
    files = []
    for f in os.listdir(d):
        files.append(d + f)
    return files

def xr_files(alpha, eps, cluster=False):
    files = []
    for d in integration_directories(alpha, eps, cluster):
        files += list_directory(d)
    return files
    

In [ ]:
a, e = ae_pairs[0]
transition_files = xr_files(a, e)

for file in transition_files:
    ds = xr.open_dataset(file)
    print(ds)
    ds.close()

In [ ]:
# Saving transitions

In [10]:
def transition_parent_dir(cluster=False):
    if cluster:
        return # WHERE WILL SAVE TRANSITION ON CLUSTER
    else:
        return '/Users/cfn18/Desktop/Double-Well-SR/Transition-Info/'
    
def transition_dir(alpha, eps, cluster=False):
    p_dir = transition_parent_dir(cluster)
    alpha_sub_dir = f'alpha_{alpha}/'.replace('.', '_')
    eps_sub_dir = f'eps_{eps}/'.replace('.', '_')
    transition_directory = p_dir + alpha_sub_dir + eps_sub_dir
    
    directories = [transition_directory + x for x in ['cold-to-hot/', 'hot-to-cold/']]
    for d in directories:
        if not os.path.exists(d):
            os.makedirs(d)
            print(f'Made directory at {d}')
    return directories
    

In [21]:
ball_size = 0.1

for a, e in tqdm(ae_pairs):    
    
    # Initialise list for transitions for fixed a, e pair
    c2h_transitions = []
    h2c_transtions = []
    
    # Loop through hot/cold ensemble files
    integration_files = xr_files(a, e)
    for file in integration_files:
        ds = xr.open_dataset(file)
        c2h, h2c = get_transitions(ds, ball_size)
        c2h_transitions += c2h
        h2c_transtions += h2c
    
    # Save Transitions
    c2h_save_dir, h2c_save_dir = transition_dir(a, e)
    save_list(c2h_transitions[:10], c2h_save_dir)
    save_list(c2h_transitions[:10], h2c_save_dir)
        

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

Saved list at /Users/cfn18/Desktop/Double-Well-SR/Transition-Info/alpha_0_0/eps_10_0/cold-to-hot/
Saved list at /Users/cfn18/Desktop/Double-Well-SR/Transition-Info/alpha_0_0/eps_10_0/hot-to-cold/


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

Saved list at /Users/cfn18/Desktop/Double-Well-SR/Transition-Info/alpha_0_0/eps_1_0/cold-to-hot/
Saved list at /Users/cfn18/Desktop/Double-Well-SR/Transition-Info/alpha_0_0/eps_1_0/hot-to-cold/


In [14]:
def get_transitions(ds, ball_size=0.1):
    c2h = []
    h2c = []
    for r in tqdm(ds.realisation):
        ts = ds.sel(realisation=r).drop('realisation')
        c2h += cold_to_hot_transitions(ts, ball_size)
        h2c += hot_to_cold_transitions(ts, ball_size)
    return c2h, h2c

def save_list(ds_list, save_dir):
    for i, ds in enumerate(ds_list):
        save_name = f'{i+1}.nc'
        ds.to_netcdf(save_dir + save_name)
        ds.close()
    print(f'Saved list at {save_dir}')